# Categorical Variable Embeddings

### Load the requied libraries

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score

from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Concatenate, Flatten, Input, Reshape, Dropout

### Read the data

In [ ]:
df = pd.read_csv('cmc.data', header=None, names=['Age','Education','H_education',
                                                 'num_child','Religion', 'Employ',
                                                 'H_occupation','living_standard',
                                                 'Media_exposure','contraceptive'])
df.head()

,Age,Education,H_education,num_child,Religion,Employ,H_occupation,living_standard,Media_exposure,contraceptive
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1
3,42,3,2,9,1,1,3,3,0,1
4,36,3,3,8,1,1,3,2,0,1


In [ ]:
df.dtypes

Age                int64
Education          int64
H_education        int64
num_child          int64
Religion           int64
Employ             int64
H_occupation       int64
living_standard    int64
Media_exposure     int64
contraceptive      int64
dtype: object

### Null check: no missing record in this data

In [ ]:
df.isnull().sum()

Age                0
Education          0
H_education        0
num_child          0
Religion           0
Employ             0
H_occupation       0
living_standard    0
Media_exposure     0
contraceptive      0
dtype: int64

### Check bad records: no bad record in this data like negative age or negative num_child

In [ ]:
df.describe()

,Age,Education,H_education,num_child,Religion,Employ,H_occupation,living_standard,Media_exposure,contraceptive
count,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000
mean,32.538357,2.958588,3.429735,3.261371,0.850645,0.749491,2.137814,3.133741,0.073999,1.919891
std,8.227245,1.014994,0.816349,2.358549,0.356559,0.433453,0.864857,0.976161,0.261858,0.876376
min,16.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
25%,26.000000,2.000000,3.000000,1.000000,1.000000,0.000000,1.000000,3.000000,0.000000,1.000000
50%,32.000000,3.000000,4.000000,3.000000,1.000000,1.000000,2.000000,3.000000,0.000000,2.000000
75%,39.000000,4.000000,4.000000,4.000000,1.000000,1.000000,3.000000,4.000000,0.000000,3.000000
max,49.000000,4.000000,4.000000,16.000000,1.000000,1.000000,4.000000,4.000000,1.000000,3.000000


### Contraceptive is the target attribute and it has three levels. So this is multiclass classification problem.  

In [ ]:
df['contraceptive'].value_counts()

1    629
3    511
2    333
Name: contraceptive, dtype: int64

In [ ]:
target = df['contraceptive'].copy()
target.head()

0    1
1    1
2    1
3    1
4    1
Name: contraceptive, dtype: int64

### One-hot encoder on the target variable

In [ ]:
df = pd.get_dummies(columns=['contraceptive'], data=df, prefix_sep="_", drop_first=False)
df.head()

,Age,Education,H_education,num_child,Religion,Employ,H_occupation,living_standard,Media_exposure,contraceptive_1,contraceptive_2,contraceptive_3
0,24,2,3,3,1,1,2,3,0,1,0,0
1,45,1,3,10,1,1,3,4,0,1,0,0
2,43,2,3,7,1,1,3,4,0,1,0,0
3,42,3,2,9,1,1,3,3,0,1,0,0
4,36,3,3,8,1,1,3,2,0,1,0,0


### Education, H_education, Religion, Employ, H_occupation, living_standard, Media_exposure and Contraceptive are having fixed set of values. Even though they are number they are Categorical Attributes. 

In [ ]:
categorical_vars = ['Education', 'H_education', 'Religion', 'Employ', 'H_occupation',
                   'living_standard', 'Media_exposure']
integral_vars = ['Age', 'num_child']
target_vars = ['contraceptive_1', 'contraceptive_2', 'contraceptive_3']

In [ ]:
for categorical_var in categorical_vars:
    print(categorical_var, df[categorical_var].unique())

Education [2 1 3 4]
H_education [3 2 4 1]
Religion [1 0]
Employ [1 0]
H_occupation [2 3 1 4]
living_standard [3 4 2 1]
Media_exposure [0 1]


### Train-test split

In [ ]:
#Performing train test split on the data
X_df, y_df = df.loc[:, categorical_vars+integral_vars], df.loc[:, target_vars]
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.3, random_state=123)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1031, 9) (1031, 3) (442, 9) (442, 3)


### Categorical embeddings + numerical inputs:

In [ ]:
inputs = []
embeddings = []
for categorical_var in categorical_vars:   
    input_cat = Input(shape=(1,))
    no_of_levels = X_train[categorical_var].nunique()
    embedding_size = int(min(np.ceil(np.sqrt(no_of_levels)), 30))
    # Categorical embedding (square root of the number of levels or 30 whichever is lower)
    
    embedding = Embedding(no_of_levels+1 ,embedding_size, input_length=1)(input_cat)
    embedding = Reshape(target_shape=(embedding_size,))(embedding)
    inputs.append(input_cat)
    embeddings.append(embedding)
    
input_numeric = Input(shape=(len(integral_vars),))
embedding_numeric = Dense(5)(input_numeric) 
inputs.append(input_numeric)
embeddings.append(embedding_numeric)

### Converting data to list format to match the network structure

In [ ]:
input_list_train = []
input_list_test = []
    
#the categorical cols to be embedded: rescaling to range [0, # levels)
for categorical_var in categorical_vars:
    levels = X_train[categorical_var].unique()
    level_map = {}
    for i in range(len(levels)):
        level_map[levels[i]] = i       
    input_list_train.append(X_train[categorical_var].map(level_map).values)
    input_list_test.append(X_test[categorical_var].map(level_map).values)
     
# Standardize numerical columns:
scale = MinMaxScaler()
scale.fit(X_train[integral_vars])
input_list_train.append(scale.transform(X_train[integral_vars]))
input_list_test.append(scale.transform(X_test[integral_vars]))

### Perceptron Model Building: Multi-Class Cross-Entropy Loss and 'softmax' activation in order to predict the probability for each class.

In [ ]:
first_layer = Concatenate()(embeddings)
outputs = Dense(len(target_vars), activation='softmax')(first_layer)
perceptron_model = Model(inputs, outputs)
perceptron_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
perceptron_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
input_10 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
input_11 (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
input_12 (InputLayer)            (None, 1)             0                                            
___________________________________________________________________________________________

In [ ]:
perceptron_model.fit(input_list_train, y_train.values, epochs=100)

Epoch 1/100
1031/1031 [==============================] - 1s - loss: 1.0673     
Epoch 2/100
1031/1031 [==============================] - 0s - loss: 1.0581     
Epoch 3/100
1031/1031 [==============================] - 0s - loss: 1.0499     
Epoch 4/100
1031/1031 [==============================] - 0s - loss: 1.0415     
Epoch 5/100
1031/1031 [==============================] - 0s - loss: 1.0336     
Epoch 6/100
1031/1031 [==============================] - 0s - loss: 1.0254     
Epoch 7/100
1031/1031 [==============================] - 0s - loss: 1.0175     
Epoch 8/100
1031/1031 [==============================] - 0s - loss: 1.0113     
Epoch 9/100
1031/1031 [==============================] - 0s - loss: 1.0053     
Epoch 10/100
1031/1031 [==============================] - 0s - loss: 1.0000     
Epoch 11/100
1031/1031 [==============================] - 0s - loss: 0.9958     
Epoch 12/100
1031/1031 [==============================] - 0s - loss: 0.9920     
Epoch 13/100
1031/1031 [=============

#### Make predictions and evaluate the predictions by using accuracy, recall and F1 score

In [ ]:
test_pred = perceptron_model.predict(input_list_test)
train_pred = perceptron_model.predict(input_list_train)

y_test_pred = test_pred.argmax(axis=1) + 1
y_train_pred = train_pred.argmax(axis=1) + 1

print('Accuracy on testing data: {}'.format(accuracy_score(target.loc[y_test.index], y_test_pred)))
print('Recall on testing data: {}'.format(recall_score(target.loc[y_test.index], y_test_pred, average='macro')))
print('F1 score on testing data: {}'.format(f1_score(target.loc[y_test.index], y_test_pred, average='macro')))

print('Accuracy on training data: {}'.format(accuracy_score(target.loc[y_train.index], y_train_pred)))
print('Recall on training data: {}'.format(recall_score(target.loc[y_train.index], y_train_pred, average='macro')))
print('F1 score on training data: {}'.format(f1_score(target.loc[y_train.index], y_train_pred, average='macro')))

Accuracy on testing data: 0.5271493212669683
Recall on testing data: 0.48971082164273305
F1 score on testing data: 0.49101335295504206
Accuracy on training data: 0.5169738118331717
Recall on training data: 0.487140673517078
F1 score on training data: 0.4916821562459728


### Add one hidden layer

In [ ]:
embedding_layer = Concatenate()(embeddings)
embedding_layer_dropout = Dropout(0.1)(embedding_layer)
hidden_layer = Dense(100, activation='relu')(embedding_layer_dropout)                                   
hidden_layer_dropout = Dropout(0.2)(hidden_layer)                                  
outputs_with_1_hidden_layer = Dense(len(target_vars), activation='softmax')(hidden_layer_dropout)
one_hidden_layer_model = Model(inputs, outputs_with_1_hidden_layer)
one_hidden_layer_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
one_hidden_layer_model.fit(input_list_train, y_train.values, epochs=100)

Epoch 1/100
1031/1031 [==============================] - 4s - loss: 1.0712     
Epoch 2/100
1031/1031 [==============================] - 0s - loss: 1.0121     
Epoch 3/100
1031/1031 [==============================] - 0s - loss: 0.9783     
Epoch 4/100
1031/1031 [==============================] - 0s - loss: 0.9601     
Epoch 5/100
1031/1031 [==============================] - 0s - loss: 0.9500     
Epoch 6/100
1031/1031 [==============================] - 0s - loss: 0.9396     
Epoch 7/100
1031/1031 [==============================] - 0s - loss: 0.9373     
Epoch 8/100
1031/1031 [==============================] - 0s - loss: 0.9252     
Epoch 9/100
1031/1031 [==============================] - 0s - loss: 0.9206     
Epoch 10/100
1031/1031 [==============================] - 0s - loss: 0.9118     
Epoch 11/100
1031/1031 [==============================] - 0s - loss: 0.9203     
Epoch 12/100
1031/1031 [==============================] - 0s - loss: 0.9128     
Epoch 13/100
1031/1031 [=============

#### Evaluate predictions

In [ ]:
test_pred = one_hidden_layer_model.predict(input_list_test)
train_pred = one_hidden_layer_model.predict(input_list_train)

y_test_pred = test_pred.argmax(axis=1) + 1
y_train_pred = train_pred.argmax(axis=1) + 1

print('Accuracy on testing data: {}'.format(accuracy_score(target.loc[y_test.index], y_test_pred)))
print('Recall on testing data: {}'.format(recall_score(target.loc[y_test.index], y_test_pred, average='macro')))
print('F1 score on testing data: {}'.format(f1_score(target.loc[y_test.index], y_test_pred, average='macro')))

print('Accuracy on training data: {}'.format(accuracy_score(target.loc[y_train.index], y_train_pred)))
print('Recall on training data: {}'.format(recall_score(target.loc[y_train.index], y_train_pred, average='macro')))
print('F1 score on training data: {}'.format(f1_score(target.loc[y_train.index], y_train_pred, average='macro')))

Accuracy on testing data: 0.581447963800905
Recall on testing data: 0.5609240476959919
F1 score on testing data: 0.562087052226994
Accuracy on training data: 0.6488845780795345
Recall on training data: 0.6306758888219562
F1 score on training data: 0.6311072166758441
